## 1. 创建 FP 树的数据结构

In [ ]:
class treeNode:
    def __init__(self, name, num, parentNode):
        self.name = name
        self.num = num
        self.parent = parentNode
        self.nodeLink = None
        self.children = {}

## 2. 构建 FP 树

In [ ]:
def createTree(dataSet, minSup):
    item_count = {}#统计各项出现次数
    #第一次遍历，得到频繁一项集
    # TODO
    headerTable={}
    #剔除不满足最小支持度的项
    # TODO
    
    freqItemSet = set(headerTable.keys())#满足最小支持度的频繁项集
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] # element: [count, node]
    
    tree_header = treeNode('Null',1,None)
    #第二次遍历，建树
    # TODO
    return tree_header, headerTable

def updateTree(items, node, headerTable):
    if items[0] in node.children:
        # 判断items的第一个结点是否已作为子结点
        node.children[items[0]].num+=1
    else:
        # 创建新的分支
        node.children[items[0]] = treeNode(items[0],1,node)
        # 更新相应频繁项集的链表，往后添加
        # TODO
    # 递归
    if len(items) > 1:
        updateTree(items[1:], node.children[items[0]], headerTable)

def updateHeader(node, targetNode):#更新headertable中的node节点形成的链表
    while node.nodeLink != None:
        node = node.nodeLink
    node.nodeLink = targetNode


In [ ]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat
 
def createInitSet(dataSet):
    retDict = {}
    for t in dataSet:
        retDict[frozenset(t)] = 1
    return retDict

In [ ]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
initSet

In [ ]:
myFPtree, myHeaderTab = createTree(initSet, 3)

## 3. 抽取条件模式基
- 发现以给定元素项结尾的所有路径的函数

In [ ]:
def find_cond_pattern_base(node_name, headerTable):
    treeNode = headerTable[node_name][1]
    cond_pat_base = {}
    # TODO
    return cond_pat_base

def find_path(node, nodepath):
    # 迭代上溯整棵树
    if node.parent != None:
        nodepath.append(node.parent.name)
        find_path(node.parent, nodepath)

In [ ]:
find_cond_pattern_base('x', myHeaderTab)

In [ ]:
find_cond_pattern_base('z', myHeaderTab)

In [ ]:
find_cond_pattern_base('r', myHeaderTab)

In [ ]:
find_cond_pattern_base('t', myHeaderTab)

## 4. 创建条件 FP 树

In [ ]:
def create_cond_fptree(headerTable, min_support, temp, freq_items,support_data):
    # 最开始的频繁项集是headerTable中的各元素
    freqs = [v[0] for v in sorted(headerTable.items(), key=lambda p:p[1][0])] # 根据频繁项的总频次排序
    for freq in freqs: # 对每个频繁项
        freq_set = temp.copy()
        freq_set.add(freq)
        freq_items.add(frozenset(freq_set))
        if frozenset(freq_set) not in support_data:#检查该频繁项是否在support_data中
            support_data[frozenset(freq_set)]=headerTable[freq][0]
        else:
            support_data[frozenset(freq_set)]+=headerTable[freq][0]

        cond_pat_base = find_cond_pattern_base(freq, headerTable)#寻找到所有条件模式基
        cond_pat_dataset=[]#将条件模式基字典转化为数组
        for item in cond_pat_base:
            item_temp=list(item)
            item_temp.sort()
            for i in range(cond_pat_base[item]):
                cond_pat_dataset.append(item_temp)
        #创建条件模式树
        cond_tree, cur_headtable = createTree(cond_pat_dataset, min_support)
        if cur_headtable != None:
            create_cond_fptree(cur_headtable, min_support, freq_set, freq_items,support_data) # 递归挖掘条件FP树


In [ ]:
def generate_L(data_set,min_support):
    freqItemSet=set()
    support_data={}
    _, headerTable=createTree(data_set,min_support)
    #创建各频繁一项的fptree，并挖掘频繁项并保存支持度计数
    create_cond_fptree(headerTable, min_support, set(), freqItemSet,support_data)
    
    max_l=0
    for i in freqItemSet:#将频繁项根据大小保存到指定的容器L中
        if len(i)>max_l:max_l=len(i)
    L=[set() for _ in range(max_l)]
    for i in freqItemSet:
        L[len(i)-1].add(i)
    for i in range(len(L)):
        print("frequent item {}:{}".format(i+1,len(L[i]))) 
    return L,support_data 


In [ ]:
generate_L(initSet, 3)